This notebook considers a famous wooly mammoth dataset, which has been used to show the types of artifacts that can be introduced by UMAP [[1](https://pair-code.github.io/understanding-umap/), [2](https://www.maxnoichl.eu/projects/mammoth/)]. Each row of the data set below corresponds to a readout from a 3D point cloud scan of a woolly mammoth skeleton. Since the data are only three-dimensional, we can visualize the raw data. Nonetheless for being relatively low dimensional, it can still be challenging for UMAP to both (1) preserve the global geometry of the skeleton and (2) capture subtleties in the relationship between individual bones. 

In [ ]:
import numpy as np
import pandas as pd

data = pd.read_json("https://raw.githubusercontent.com/krisrs1128/distortions-data/main/data/mammoth_3d.json")
data.columns = ["x", "y", "z"]

## Data Preparation

We first run a new map with the 50 nearest neighbor graph. We are using the scanpy package just for consistency with the other notebooks. In principle, we could've also used the ordinary umap package.

In [ ]:
from anndata import AnnData
import scanpy as sc

n_neighbors = 50
adata = AnnData(X=data, obs=pd.DataFrame(index=data.index))
sc.pp.neighbors(adata, n_neighbors=n_neighbors)
sc.tl.umap(adata)
embedding = adata.obsm["X_umap"].copy()

In the block below, we estimate the local distortion associated with each point in the point cloud. This estimation step hinges on the geometry object. This is exactly like the geometry objects defined in the megaman package, and it is used to define the intrinsic geometry of the manifold from which the data are assumed to be sampled. This depends on hyperparameters. We have set the number of neighbors to equal the number of neighbors in UMAP, and we have set the radius to be a small multiple of the typical neighborhood distance. We have found these to be reasonable defaults, but still encourage you to test sensitivity, especially to the radius and $\epsilon$ scaling parameter.

In [3]:
from distortions.geometry import Geometry, bind_metric, local_distortions

radius = 3 * np.mean(adata.obsp["distances"].data)
geom = Geometry("brute", laplacian_method="geometric", affinity_kwds={"radius": radius}, adjacency_kwds={"n_neighbors": n_neighbors}, laplacian_kwds={"scaling_epps": 5})
H, Hvv, Hs = local_distortions(embedding, data, geom)
embedding = bind_metric(embedding, Hvv, Hs)

## Static Visualizations

In [13]:
import plotly.express as px

fig = px.scatter_3d(data, x='x', y='z', z='y')
fig.update_traces(marker_size=1, marker_color="black")
fig

In [14]:
import altair as alt
alt.data_transformers.enable("vegafusion")

alt.Chart(embedding).mark_circle(color="black").encode(
    x=alt.X("embedding_0"),
    y=alt.Y("embedding_1")
).properties(width=450, height=350)

alt.Chart(...)

In [15]:
from distortions.geometry import neighborhood_distances

distances = neighborhood_distances(adata)
alt.Chart(distances).mark_boxplot(outliers=True).encode(
    x=alt.X('true', bin={"maxbins": 20}),
    y=alt.Y('embedding')
)

alt.Chart(...)

## Interactive Visualizations

In [7]:
from distortions.geometry import neighborhoods
from distortions.visualization import dplot

plots = {}
N = neighborhoods(adata, threshold=.1, outlier_factor=3)
plots["mamoth_links"] = dplot(embedding, height = 350, width=450)\
    .mapping(x="embedding_0", y="embedding_1")\
    .inter_edge_link(N=N, strokeWidth=.2, opacity=0.7, threshold=4, stroke="#F25E7A", highlightColor="#C83F58")\
    .geom_ellipse(opacity=0.8, radiusMin=.5, radiusMax=10)

In [8]:
metrics = {k: H[k] for k in range(len(H))}
plots["mammoth_isometry"] = dplot(embedding, height=350, width=450)\
    .mapping(x="embedding_0", y="embedding_1")\
    .inter_isometry(metrics=metrics, transformation_bw=.5, metric_bw=.1)\
    .geom_ellipse(opacity=0.8, radiusMin=.5, radiusMax=10)

In [9]:
plots["mammoth_box"] = dplot(embedding, height=350, width=530)\
    .mapping(x="embedding_0", y="embedding_1")\
    .geom_ellipse(radiusMax=10, radiusMin=.5)\
    .inter_boxplot(dists=distances, outlier_iqr=10, highlightColor="#F25E7A", strokeWidth=0.4)

In [10]:
[display(p) for p in plots.values()]

dplot(dataset=[{'embedding_0': 8.870267868041992, 'embedding_1': 0.8856074810028076, 'x0': -0.8852808564862759…

dplot(dataset=[{'embedding_0': 8.870267868041992, 'embedding_1': 0.8856074810028076, 'x0': -0.8852808564862759…

dplot(dataset=[{'embedding_0': 8.870267868041992, 'embedding_1': 0.8856074810028076, 'x0': -0.8852808564862759…

[None, None, None]

In [11]:
#[p.save(f"../paper/figures/{k}.svg") for k, p in plots.items()]

In [12]:
len(N)

425